In [ ]:
from pypdf import PdfReader

In [ ]:
pdf_reader = PdfReader('./Civics.pdf')

In [ ]:
data = []
for i, page in enumerate(pdf_reader.pages):
    data.append({'page':i, 'content':page.extract_text()})

In [ ]:
data = data[6:len(pdf_reader.pages)]

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(data)

In [ ]:
import nltk
import string
pd.options.mode.chained_assignment = None

In [ ]:
df['text_lower'] = df['content'].str.lower()

In [ ]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('','', PUNCT_TO_REMOVE))

df['text_wo_punct'] = df['text_lower'].apply(lambda text: remove_punctuation(text))


In [ ]:
import chromadb
import sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
documents = []
metadatas = []
ids = []

for i in range(len(df)):
    documents.append(df['text_wo_punct'][i][6:])
    metadatas.append({'unit':df['text_wo_punct'][i][:6], 'page':i + 1})
    ids.append(str(i + 3))


In [ ]:
client = chromadb.Client()

In [ ]:
vector_coll = client.create_collection('civics-latest-1.0.0')

vector_coll.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

In [ ]:
results = vector_coll.query(
    query_texts=['equality'],
    n_results=5
)
for i in range(len(results)):
    print('Ids:' , results['ids'][0][i])
    print('Metadatas:', results['metadatas'][0][i])
    print('Description:', results['documents'][0][i])
    print('\n')